<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E9%9D%92%E7%A9%BA%E6%96%87%E5%BA%AB%E3%81%AE%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 青空文庫のテキストデータを取得する (2)

*   履歴
    *   前処理を追加しました（2025/10/14）

*   「▶」ボタンで最初のコードセルを実行してください。表が表示されたら，表の右上の「Filter」ボタンを押して絞り込み検索をして目的の作品Noを見つけてください。


In [ ]:
#@title 青空文庫の作家別作品リストを読み込む

from pathlib import Path
import pandas as pd
from google.colab import data_table

if not Path('list_person_all_utf8.csv').exists():
  !curl -RL https://www.aozora.gr.jp/index_pages/list_person_all_utf8.zip | funzip > list_person_all_utf8.csv

df_list_person = pd.read_csv('list_person_all_utf8.csv')

data_table.DataTable(df_list_person, num_rows_per_page=10)

*   目的の作品IDが分かったら，下のフォームの「作品ID」欄に入力してください。

In [ ]:
#@title 作品IDを指定してファイルを取得する

from pathlib import Path
import re

import pandas as pd


def aozora_load_filelist():
  path = 'filelist.csv'

  if not Path(path).exists():
    url = 'https://github.com/dfukagaw28/ColabNotebooks/raw/refs/heads/main/aozora/filelist.csv'
    !curl -RLO {url}

  df_filelist = pd.read_csv(path)

  return df_filelist


def aozora_download_text(work_info, force=False):
  zip_path = work_info['path']
  zip_path = Path(zip_path)
  text_path = (zip_path.with_suffix('') / zip_path.name).with_suffix('.txt')
  text_url = 'https://github.com/dfukagaw28/aozorabunko_text/raw/refs/heads/master/' + str(text_path)

  author_id = work_info['author_id']
  author_id = f'000000{author_id}'[-6:]
  text_filename = work_info['text_filename']

  output_path = Path(str(author_id)) / text_filename

  if force or not output_path.exists():
    output_path.parent.mkdir(parents=True, exist_ok=True)
    !curl -RL {text_url} -o {output_path}
    !mv {output_path} {output_path}.orig
    with open(f'{output_path}.orig', encoding='cp932') as file:
      text = file.read()
      text = aozora_preprocess(text)
    with open(output_path, 'w', encoding='utf_8') as file:
      file.write(text)
    !rm -f {output_path}.orig

  return output_path


def aozora_preprocess(text):
  # 冒頭の記載事項（タイトル・著者名等）を検出して除去する
  try:
    pos = text.index('\n\n')
    text = text[pos+2:]
  except:
    pass

  # 【テキスト中に現れる記号について】を検出して除去する
  try:
    # 最初の行が「--------」で始まることを確認する
    assert text.startswith('--------')

    # ２つ目の「--------」で始まる行と，その次の空行を除去する
    pos = text.index('\n') + 1
    pos = text.index('--------', pos)
    pos = text.index('\n\n', pos) + 2
    text = text[pos:]
  except:
    pass

  # 本文の末尾の情報（底本，入力・校正にかんする情報等）を検出して除去する
  try:
    pos = text.index('\n\n\n\n')
    text = text[:pos]
  except:
    pass

  # ルビに関する記述を削除する
  text = re.sub('《.+?》', '', text)
  text = re.sub('｜', '', text)

  # 注記情報（レイアウト情報，入力者注等）を検出して削除する
  text = re.sub(r'［＃.*?］', '', text)

  # 文ごとに改行する
  text = re.sub('(。|？)', '\\1\n', text)

  # HORIZONTAL BAR (U+2015) を EM DASH (U+2014) に変換する
  text = text.replace('\u2015', '\u2014')

  return text


作品ID = 789  #@param {"type":"integer"}

df_filelist = aozora_load_filelist()

df_filtered = df_filelist[df_filelist['work_id'] == 作品ID]

if len(df_filtered) == 1:
  # 作品に関する情報を取得
  row = df_filtered.iloc[0]

  # 作品のテキストファイルをダウンロードする
  # 既にファイルが存在する場合はダウンロードを省略する
  output_path = aozora_download_text(row, force=True)

  # ダウンロードしたテキストファイルのパス
  print(output_path)

  # テキストファイルを開く
  from google.colab import files
  files.view(output_path)

  # 自動的にダウンロードする
  files.download(output_path)
